In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder


In [8]:
df = pd.read_csv("spam.csv", encoding='latin-1', usecols = ['v2', 'v1'])

In [11]:
df['v2'] = df['v2'].apply(lambda x: x.lower()).apply(lambda x: re.sub('[^a-zA-Z0-9\s]','',x))
df.head()

,v1,v2
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...


In [12]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures)
tokenizer.fit_on_texts(df['v2'].values)
X = tokenizer.texts_to_sequences(df['v2'].values)
X = pad_sequences(X)


In [13]:
embed_dim = 128
lstm_out = 196


In [14]:
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # print(model.summary())
    return model

In [18]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(df['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train,  epochs = 5, batch_size=batch_size, verbose=2)
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
print(model.metrics_names[0], ":", score)
print(model.metrics_names[1], ":", acc)

Epoch 1/5
117/117 - 72s - loss: 0.1506 - accuracy: 0.9480
Epoch 2/5
117/117 - 74s - loss: 0.0408 - accuracy: 0.9879
Epoch 3/5
117/117 - 68s - loss: 0.0176 - accuracy: 0.9941
Epoch 4/5
117/117 - 69s - loss: 0.0155 - accuracy: 0.9954
Epoch 5/5
117/117 - 69s - loss: 0.0071 - accuracy: 0.9981
58/58 - 4s - loss: 0.0905 - accuracy: 0.9772
loss : 0.09049486368894577
accuracy : 0.9771615266799927
